In [1]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from tensorflow.keras import utils
#import play

In [2]:
from music21 import converter, instrument, note, chord, stream

In [3]:
songs = glob('Jazz/*.mid')

In [4]:
songs = songs[:3]

In [5]:
def get_notes():
    notes = []
    for file in songs:
        # converting .mid file to stream object
        midi = converter.parse(file)
        notes_to_parse = []
        try:
            # Given a single stream, partition into a part for each unique instrument
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts: # if parts has instrument parts 
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
    
        for element in notes_to_parse: 
            if isinstance(element, note.Note):
                # if element is a note, extract pitch
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                # if element is a chord, append the normal form of the 
                # chord (a list of integers) to the list of notes. 
                notes.append('.'.join(str(n) for n in element.normalOrder))
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

In [6]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    # Extract the unique pitches in the list of notes.
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, LSTM, Dropout, Flatten
def create_network(network_in, n_vocab): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'weights.best.music3.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

In [9]:
def train_network():
    """
    Get notes
    Generates input and output sequences
    Creates a model 
    Trains the model for the given epochs
    """
    
    epochs = 200
    
    notes = get_notes()
    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated')
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')
    
    return model

In [10]:
### Train the model 
train_network()

Notes processed
Vocab generated
Input and Output processed
Model created
Training in progress
Epoch 1/200
6/6 [==============================] - 8s 221ms/step - loss: 2.8811
Epoch 2/200
6/6 [==============================] - 1s 213ms/step - loss: 2.6398
Epoch 3/200
6/6 [==============================] - 1s 190ms/step - loss: 2.6277
Epoch 4/200
6/6 [==============================] - 1s 191ms/step - loss: 2.5924
Epoch 5/200
6/6 [==============================] - 1s 193ms/step - loss: 2.4994
Epoch 6/200
6/6 [==============================] - 1s 190ms/step - loss: 2.4495
Epoch 7/200
6/6 [==============================] - 1s 192ms/step - loss: 2.3983
Epoch 8/200
6/6 [==============================] - 1s 192ms/step - loss: 2.3794
Epoch 9/200
6/6 [==============================] - 1s 192ms/step - loss: 2.2846
Epoch 10/200
6/6 [==============================] - 1s 192ms/step - loss: 2.1618
Epoch 11/200
6/6 [==============================] - 1s 208ms/step - loss: 2.0885
Epoch 12/200
6/6 [======

6/6 [==============================] - 1s 191ms/step - loss: 0.0021
Epoch 101/200
6/6 [==============================] - 1s 203ms/step - loss: 0.0030
Epoch 102/200
6/6 [==============================] - 1s 208ms/step - loss: 7.1022e-04
Epoch 103/200
6/6 [==============================] - 1s 219ms/step - loss: 0.0018
Epoch 104/200
6/6 [==============================] - 1s 208ms/step - loss: 0.0011 0s - loss: 0.00
Epoch 105/200
6/6 [==============================] - 1s 196ms/step - loss: 6.6024e-04
Epoch 106/200
6/6 [==============================] - 1s 191ms/step - loss: 0.0020
Epoch 107/200
6/6 [==============================] - 1s 192ms/step - loss: 9.1215e-04
Epoch 108/200
6/6 [==============================] - 1s 192ms/step - loss: 9.0682e-04
Epoch 109/200
6/6 [==============================] - 1s 192ms/step - loss: 4.1549e-04
Epoch 110/200
6/6 [==============================] - 1s 191ms/step - loss: 4.7468e-04
Epoch 111/200
6/6 [==============================] - 1s 193ms/step - los

6/6 [==============================] - 1s 199ms/step - loss: 7.7468e-05
Epoch 195/200
6/6 [==============================] - 1s 193ms/step - loss: 1.1061e-04
Epoch 196/200
6/6 [==============================] - 1s 191ms/step - loss: 8.3911e-05
Epoch 197/200
6/6 [==============================] - 1s 190ms/step - loss: 1.0135e-04
Epoch 198/200
6/6 [==============================] - 1s 193ms/step - loss: 8.8535e-05
Epoch 199/200
6/6 [==============================] - 1s 195ms/step - loss: 5.1576e-05
Epoch 200/200
6/6 [==============================] - 1s 193ms/step - loss: 1.0392e-04
Training completed


In [11]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    normalized_input = network_input / float(n_vocab)
    model = create_network(normalized_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('weights.best.music3.hdf5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [12]:
def get_inputSequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    
    network_input = np.reshape(network_input, (len(network_input), 100, 1))
    
    return (network_input)

In [13]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # pick a random sequence from the input as a starting point for the prediction
    pattern = list(network_input[start])
    prediction_output = []
    
    print('Generating notes........')

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.array(pattern,dtype=np.float)
        prediction_input = np.reshape(prediction_input, (1, len(pattern), 1))
        #print("The type of the prediction input : ",type(prediction_input))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        
        # Predicted output is the argmax(P(h|D))
        index = np.argmax(prediction)
        # Mapping the predicted interger back to the corresponding note
        result = int_to_note[index]
        # Storing the predicted output
        prediction_output.append(result)

        pattern.append(index)
        # Next input to the model
        pattern = pattern[1:len(pattern)]

    print('Notes Generated...')
    return prediction_output

In [14]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='test_output4.mid')

In [15]:
#### Generate a new jazz music 
generate()

Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........


C:\Users\hisha\AppData\Local\Temp\ipykernel_2976\2204820834.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction_input = np.array(pattern,dtype=np.float)
C:\Users\hisha\AppData\Local\Temp\ipykernel_2976\2204820834.py:16: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  prediction_input = np.array(pattern,dtype=np.float)


Notes Generated...
Saving Output file as midi....
